## Imports

In [20]:
import pypsa
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import os


## Read network file

In [21]:
# Function to read in the network file

def read_network_file():
    network = 'C:/Users/noraky/Documents/Test_4/pypsa-earth/networks/elec_s_all_ec_lcopt_Co2L-1H.nc'
    n = pypsa.Network(network)
    return n

network = 'C:/Users/noraky/Documents/Test_4/pypsa-earth/networks/elec_s_all_ec_lcopt_Co2L-1H.nc'
n = pypsa.Network(network)

# folder where the results from the previous year is saved
#scen_folder = 'results_base'
#scen = 'B_27'
                                                                                                                                                                                                                                                                                                                               
# # Next year of the run
#year = 2022



c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with

## Scaling factors and Decomissioning

The scaling factors are used to increase the demand and decrease the emission constraint. The decomissioning (and comissioning, if applicable) is added on a yearly basis as well.

In [16]:
# Demand scaling factor - based on the initial demand in 2021

scale_demand = {
    2022: 1.039555791,
    2023: 1.037440228,
    2024: 1.030106894,
    2025: 1.029322163,
    2026: 1.051886792,
    2027: 1.044491339,
    2028: 1.044448186,
    2029: 1.044491013,
    2030: 1.044447874,
    2031: 1.044477281,
    2032: 1.044422438,
    2033: 1.040027091,
    2034: 1.039984371,
    2035: 1.040012523,
    'back':0.569269673, # scaling factor for scaling from 2035 to 2021
    'direct' : 1.7566367 # scaling factor from 2021 to 2035
}


In [4]:
# emissions decline

start_year = 2021
start_value = 2370000
end_year = 2035
end_value = 0

years = range(start_year, end_year + 1)
emissions = []

decrease_per_year = (start_value - end_value) / (end_year - start_year)

for i in years:
    emission = max(start_value - decrease_per_year * (i - start_year), end_value)
    emissions.append(emission)

# Make into dict
index = np.arange(2021,2036)
emission_limit = dict(zip(index, emissions))
emission_limit


{2021: 2370000.0,
 2022: 2200714.285714286,
 2023: 2031428.5714285714,
 2024: 1862142.8571428573,
 2025: 1692857.1428571427,
 2026: 1523571.4285714286,
 2027: 1354285.7142857143,
 2028: 1185000.0,
 2029: 1015714.2857142857,
 2030: 846428.5714285714,
 2031: 677142.857142857,
 2032: 507857.1428571427,
 2033: 338571.42857142864,
 2034: 169285.7142857141,
 2035: 0.0}

In [5]:
# decomissioning of powerplants

decom = {
2022 : {'BO 0 OCGT':0, 'BO 1 OCGT':20.3, 'BO 2 OCGT': 36.76, 'BO 3 oil':0},
2023 : {'BO 0 OCGT':0, 'BO 1 OCGT':20.72, 'BO 2 OCGT': 36.77, 'BO 3 oil':0},
2024 : {'BO 0 OCGT':0, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2025 : {'BO 0 OCGT':25.44, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2026 : {'BO 0 OCGT':0, 'BO 1 OCGT':17.86, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2027 : {'BO 0 OCGT':0, 'BO 1 OCGT':105.87, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2028 : {'BO 0 OCGT':0, 'BO 1 OCGT':16.9, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2029 : {'BO 0 OCGT':0, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2030 : {'BO 0 OCGT':0, 'BO 1 OCGT':97.85, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2031 : {'BO 0 OCGT':0, 'BO 1 OCGT':109.37, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2032 : {'BO 0 OCGT':0, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2033 : {'BO 0 OCGT':0, 'BO 1 OCGT':20.72, 'BO 2 OCGT': 0, 'BO 3 oil':2.16},
2034 : {'BO 0 OCGT':0, 'BO 1 OCGT':20.72, 'BO 2 OCGT': 0, 'BO 3 oil':0},
2035 : {'BO 0 OCGT':0, 'BO 1 OCGT':0, 'BO 2 OCGT': 0, 'BO 3 oil':0},
}

# To force expansion, this dicts is used
expansion = {
    2022 : {'BO 0 geothermal':5, 'BO 0 solar':13, 'BO 2 ror':126},
    2023: { 'BO 2 solar':13, 'BO 3 ror':126},
    2024: {'BO 1 biomass': 21, 'BO 1 onwind':53, 'BO 0 solar':13, 'BO 2 ror':126},
    2025: { 'BO 1 onwind':24,'BO 2 solar':13, 'BO 3 ror':126},
    2026: {'BO 3 biomass': 21, 'BO 0 solar':13, 'BO 2 ror':126},
    2027: {'BO 0 geothermal':45, 'BO 1 onwind':25,'BO 2 solar':13, 'BO 3 ror':126},
    2028: {'BO 1 biomass': 21, 'BO 1 onwind':25, 'BO 0 solar':13, 'BO 2 ror':126},
    2029: {'BO 2 solar':13, 'BO 3 ror':126},
    2030: {'BO 3 biomass': 21, 'BO 1 onwind':35, 'BO 0 solar':13, 'BO 2 ror':126},
    2031: {},
    2032:{},
    2033:{},
    2034:{},
    2035:{},
}


# Nodes in the model (can change!!!!)
#0 : Southern
# 1: Oriental
#2: Central
#3: Northern

In [11]:
n.generators

#
# OCGT -> CCGT

,p_nom_max,weight,p_nom,capital_cost,efficiency,p_nom_min,marginal_cost,bus,carrier,control,...,start_up_cost,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down
Generator,,,,,,,,,,,,,,,,,,,,,
10 onwind,1.554366e+03,10.011657,0.00,109295.569252,1.0,0.0,0.015,10,onwind,Slack,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
10 solar,2.686871e+03,481.564185,0.00,50855.958267,1.0,0.0,0.010,10,solar,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
21 onwind,2.955995e+04,490.343900,0.00,109295.569252,1.0,0.0,0.015,21,onwind,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
21 ror,inf,1.890000,1.89,270940.715283,0.9,0.0,0.000,21,ror,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
21 solar,4.209422e+04,7880.946130,0.00,50855.958267,1.0,0.0,0.010,21,solar,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73 solar,4.489479e+03,757.116362,0.00,50855.958267,1.0,0.0,0.010,73,solar,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
89 onwind,2.569871e+03,128.600148,0.00,109295.569252,1.0,0.0,0.015,89,onwind,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0
89 solar,4.001768e+03,790.759928,0.00,50855.958267,1.0,0.0,0.010,89,solar,,...,0.0,0.0,0,0,1,0,NaN,NaN,1.0,1.0


## All changes

This cell makes all changes that need to be done to run the next year all at once and saves it in a new file. 

In [18]:
# Function that implements all yearly changes

def yearly_changes(n,year):
    print(year)
    # ------- DEMAND ---------
    upscaling_factor = scale_demand[year]
    n.loads_t.p_set = n.loads_t.p_set * upscaling_factor
    display(round((n.loads_t.p_set.sum().sum()/1000000),4))

    # ------- EMISSIONS -------
    #n.global_constraints.constant = emission_limit[year]
    #display(n.global_constraints.constant)



    # ------- GENERATOR EXTENSTION -----
    solved_network = f'C:/Users/noraky/Documents/Test_4/pypsa-earth/{scen_folder}/{scen}_{year-1}.nc'
    m = pypsa.Network(solved_network)
    additional_exp= m.generators.p_nom_opt - m.generators.p_nom
    # replace negative values with 0
    for index, value in additional_exp.items():
        if value < 0:
            additional_exp[index]=0
    # add expansion to previous network
    n.generators.p_nom = n.generators.p_nom.add(additional_exp, fill_value=0)
    n.generators.p_nom_min = n.generators.p_nom_min.add(additional_exp, fill_value=0)
    #display(n.generators.p_nom)


    # ------- STORES ----------
    additional_stores = m.stores.e_nom_opt - m.stores.e_nom
    additional_stores
    n.stores.e_nom = n.stores.e_nom.add(additional_stores, fill_value = 0)
    n.stores.e_nom_min = n.stores.e_nom_min.add(additional_stores, fill_value = 0)
    #display(n.stores.e_nom.sum())


    # ------- LINKS -------
    additional_links = m.links.p_nom_opt - m.links.p_nom
    additional_links
    n.links.p_nom = n.links.p_nom.add(additional_links, fill_value = 0)
    n.links.p_nom_min = n.links.p_nom_min.add(additional_links, fill_value = 0)
    #display(n.links.p_nom.sum())


    # ------- DECOM ---------
    #for index,value in decom[year].items():
    #    n.generators.loc[index, 'p_nom'] = n.generators.loc[index].p_nom - value
    #    n.generators.loc[index, 'p_nom_min'] = n.generators.loc[index].p_nom_min - value

    # -------- EXPANSION ----------
    # for index,value in expansion[year].items():
    #     n.generators.loc[index, 'p_nom_min'] = n.generators.loc[index].p_nom_min + value

    # ------- HYDRO-DECOM-------- # is done seperately, because the  code is different
    #if year == 2032:
    #    n.storage_units.loc['BO 2 hydro','p_nom'] = n.storage_units.loc['BO 2 hydro'].p_nom - 2.55

    # ------- SAVING ----------
    n.export_to_netcdf('C:/Users/noraky/Documents/Test_4/pypsa-earth/networks/elec_s_all_ec_lcopt_Co2L-1H.nc')

## Snakemake process

In [19]:
# Function that renames the results file and moves it to the scenario folder

def rename_results_file(year):
    current_location = 'C:/Users/noraky/Documents/Test_4/pypsa-earth/results/networks/'
    old_filename = "elec_s_all_ec_lcopt_Co2L-1H.nc"
    new_filename = f"{scen}_{year}.nc"
    new_location = f'C:/Users/noraky/Documents/Test_4/pypsa-earth/{scen_folder}/'

    # Create the full paths for the old and new files
    old_file_path = os.path.join(current_location, old_filename)
    new_file_path = os.path.join(new_location, new_filename)

    # Rename the file
    os.rename(old_file_path, new_file_path)


In [7]:
# determine scenario
scen_folder = 'results_base'
scen = 'B_27'

# !!!! Make sure the upscaling, emission constraints and decomissioning is run!!!

# run for loop over all years
for year in range(2021, 2025):
    n = read_network_file()
    if year == 2021:
        !snakemake -j 1 solve_all_networks
        rename_results_file(year)
    else:
        yearly_changes(n,year)
        !snakemake -j 1 solve_all_networks 
        rename_results_file(year)  

# And you're DONE!!! :D


c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-16
No parameters matching '_test' found
This is the repository path:  c:\Users\noraky\Documents\Test_4\pypsa-earth
Had to go 0 folder(s) up.
This is the repository path:  c:\Users\noraky\Documents\Test_4\pypsa-earth
Had to go 0 folder(s) up.
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-16
Read LP format model from file C:\Users\noraky\AppData\Local\Temp\pypsa-problem-m63uz8hn.lp
Reading time = 23.81 seconds
obj: 4879348 rows, 2295277 columns, 9791132 nonzeros
Set parameter Threads to value 4
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 1e-05
Set parameter AggFill to value 0
Set parameter PreDual to value 0
Set parameter GURO_PAR_BARDENSETHRESH to value 200
Set parameter LogFile to value "logs\solve_network\elec_s_27_ec_lcopt_Co2L-1H_solver.log"
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)



INFO:pypsa.io:Imported network elec_s_27_ec_lcopt_Co2L-1H.nc has buses, carriers, generators, global_constraints, lines, links, loads, storage_units, stores


2022


9.8291

...

2024


10.5041

c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  attrs.loc[bool_b, "default"] = attrs.loc[bool_b].isin({True, "True"})
c:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\components.py:318: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[]' has dtype incompatible with

Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-16
No parameters matching '_test' found
Set parameter Username
Academic license - for non-commercial use only - expires 2024-10-16
Read LP format model from file C:\Users\noraky\AppData\Local\Temp\pypsa-problem-6y8wos5_.lp
Reading time = 8.16 seconds
obj: 4879348 rows, 2295277 columns, 9791132 nonzeros
Set parameter Threads to value 4
Set parameter Method to value 2
Set parameter Crossover to value 0
Set parameter BarConvTol to value 1e-05
Set parameter AggFill to value 0
Set parameter PreDual to value 0
Set parameter GURO_PAR_BARDENSETHRESH to value 200
Set parameter LogFile to value "logs\solve_network\elec_s_27_ec_lcopt_Co2L-1H_solver.log"
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: 12th Gen Intel(R) Core(TM) i7-12700H, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 4 threads

Optimize a model with 4879348 rows, 2295277

C:\Users\noraky\anaconda3\envs\pypsa-earth-03102023\lib\site-packages\pypsa\networkclustering.py:16: UserWarning: The namespace `pypsa.networkclustering` is deprecated and will be removed in PyPSA v0.24. Please use `pypsa.clustering.spatial instead`. 
  warnings.warn(
Building DAG of jobs...
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Job stats:
job                   count
------------------  -------
solve_all_networks        1
solve_network             1
total                     2

job                   count
------------------  -------
solve_all_networks        1
solve_network             1
total                     2

Select jobs to execute...

INFO:snakemake.logging:
[Tue Nov 21 09:41:06 2023]
INFO:snakemake.logging:[Tue Nov 21 09:41:06 2023]
rule solve_network:
    input: networks/elec_s_27_ec_lcopt_Co2L-1H.nc
    output: results/networks/elec_s_27_ec_lcopt_Co2L-1H.nc
    log: logs\solve_network\elec_s_27_ec_lcopt_Co2L-1H

# Changes to the Network

Here all the general changes (except for expandabels, that's at the end of this file) that can be made to the model are shown. If more than the two last rules of the model are run, some of these changes may have to be done again. Use the "Checks" section to check your current network file. 

### Add Biomass in Node 0 and 3

The nodes may need to be adapted, depending on your model

In [ ]:
n.add("Generator",
            'BO 0 biomass',
            bus='BO 0',
            carrier='biomass',
            p_nom=0,
            marginal_cost=n.generators.loc['BO 1 biomass'].marginal_cost,
            capital_cost = n.generators.loc['BO 1 biomass'].capital_cost,
            efficiency = n.generators.loc['BO 1 biomass'].efficiency,
            p_nom_extendable = n.generators.loc['BO 1 biomass'].p_nom_extendable,
            p_max_pu = n.generators.loc['BO 1 biomass'].p_max_pu,
            up_time_before= 1
            
)


n.add("Generator",
            'BO 2 biomass',
            bus='BO 2',
            carrier='biomass',
            p_nom=0,
            marginal_cost=n.generators.loc['BO 1 biomass'].marginal_cost,
            capital_cost = n.generators.loc['BO 1 biomass'].capital_cost,
            efficiency = n.generators.loc['BO 1 biomass'].efficiency,
            p_nom_extendable = n.generators.loc['BO 1 biomass'].p_nom_extendable,
            p_max_pu = n.generators.loc['BO 1 biomass'].p_max_pu,
            up_time_before= 1
            
)

### Add ROR 

Nodes may vary

In [ ]:
n.storage_units

In [ ]:
n.add("Generator",
            'BO 2 ror',
            bus='BO 2',
            carrier='ror',
            p_nom=0,
            marginal_cost=0.021011,
            capital_cost = 270940.715283,
            efficiency = 0.9,
            p_nom_extendable = True,
            p_max_pu = 0.5,
            
)

n.add("Generator",
            'BO 3 ror',
            bus='BO 3',
            carrier='ror',
            p_nom=0,
            marginal_cost=0.021011,
            capital_cost = 270940.715283,
            efficiency = 0.9,
            p_nom_extendable = True,
            p_max_pu = 0.5,
            
)

### Biomas Max Capacity

The nodes may vary, the larger biomass max capacity is located in the Oriental zone

In [ ]:
for index, row in n.generators.iterrows():
    # Check if the carrier is equal to "OCGT"
    if row['carrier'] == 'biomass':
        if index == 'BO 1 biomass':
        # Update the p_max_pu value to 0.64
            n.generators.at[index, 'p_nom_max'] = 357.13
        else:
            n.generators.at[index, 'p_nom_max'] = 21.01

n.generators.p_nom_max

### Battery max capacity

In [ ]:
# discharger capacity

for index, row in n.links.iterrows():
    if row['carrier'] == 'battery discharger':
        n.links.at[index, 'p_nom_max'] = 225

display(n.links.p_nom_max)


# storage capacity

for index, row in n.stores.iterrows():
    if row['carrier'] == 'battery':
        n.stores.at[index, 'e_nom_max'] = 1350

n.stores.e_nom_max

### Capacity factors

With the new addition to model where the annual capacity factors can be set, this is not necessary anymore.

In [6]:
# Check the current status
n.generators.groupby('carrier').p_max_pu.mean()

carrier
CCGT       1.0
OCGT       1.0
biomass    1.0
oil        1.0
onwind     1.0
ror        1.0
solar      1.0
Name: p_max_pu, dtype: float64

In [ ]:
# Iterate through the rows of the DataFrame
for index, row in n.generators.iterrows():
    # Check if the carrier is equal to "OCGT"
    if row['carrier'] == 'OCGT':
        # Update the p_max_pu value to 0.64
        n.generators.at[index, 'p_max_pu'] = 0.64 # 0.08 #0.64
    if row['carrier'] == 'CCGT':
        # Update the p_max_pu value to 0.64
        n.generators.at[index, 'p_max_pu'] = 0.64 #0.45 #0.64

    if row['carrier'] == 'biomass':
        # Update the p_max_pu value to 0.72
        n.generators.at[index, 'p_max_pu'] = 0.72 #0.32 #0.72 for the future 

        
n.generators.groupby('carrier').p_max_pu.mean()

# Save elec_s_4_ec_lcopt_Co2L-1H file

If you made changes to network file at the beginning, they need to be saved again. This code can be used for that. 

In [10]:
n.export_to_netcdf('C:/Users/noraky/Documents/Test_4/pypsa-earth/networks/elec_s_4_ec_lcopt_Co2L-1H.nc')

INFO:pypsa.io:Exported network elec_s_27_ec_lcopt_Co2L-1H.nc has global_constraints, generators, storage_units, links, stores, loads, buses, lines, carriers


<xarray.Dataset>
Dimensions:                               (snapshots: 8760,
                                           investment_periods: 0,
                                           global_constraints_i: 1,
                                           generators_i: 73,
                                           generators_t_p_max_pu_i: 56,
                                           storage_units_i: 5,
                                           storage_units_t_inflow_i: 5,
                                           links_i: 56, stores_i: 28,
                                           loads_i: 28, loads_t_p_set_i: 28,
                                           buses_i: 56, lines_i: 52,
                                           carriers_i: 11)
Coordinates: (12/14)
  * snapshots                             (snapshots) int64 0 1 2 ... 8758 8759
  * investment_periods                    (investment_periods) int64 
  * global_constraints_i                  (global_constraints_i) object 'CO2L...
  * generators_i                          (generators_i) object 'BO0 0 OCGT' ...
  * generators_t_p_max_pu_i               (generators_t_p_max_pu_i) object 'B...
  * storage_units_i                       (storage_units_i) object 'BO0 1 hyd...
    ...                                    ...
  * stores_i                              (stores_i) object 'BO0 0 battery' ....
  * loads_i                               (loads_i) object 'BO0 0' ... 'BO1 0'
  * loads_t_p_set_i                       (loads_t_p_set_i) object 'BO0 0' .....
  * buses_i                               (buses_i) object 'BO0 0' ... 'BO1 0...
  * lines_i                               (lines_i) object '1' '2' ... '51' '52'
  * carriers_i                            (carriers_i) object 'OCGT' ... 'bat...
Data variables: (12/81)
    snapshots_snapshot                    (snapshots) datetime64[ns] 2013-01-...
    snapshots_objective                   (snapshots) float64 1.0 1.0 ... 1.0
    snapshots_generators                  (snapshots) float64 1.0 1.0 ... 1.0
    snapshots_stores                      (snapshots) float64 1.0 1.0 ... 1.0
    investment_periods_objective          (investment_periods) float64 
    investment_periods_years              (investment_periods) float64 
    ...                                    ...
    lines_s_nom_extendable                (lines_i) bool True True ... True True
    lines_s_nom_min                       (lines_i) float64 569.2 ... 853.8
    lines_build_year                      (lines_i) int64 0 0 0 0 0 ... 0 0 0 0
    carriers_co2_emissions                (carriers_i) float64 0.187 ... 0.0
    carriers_color                        (carriers_i) object '#d35050' ... '...
    carriers_nice_name                    (carriers_i) object 'Open-Cycle Gas...
Attributes:
    network_name:           
    network_pypsa_version:  0.24.0
    network_srid:           4326
    meta:                   {"version": "0.2.2", "tutorial": false, "logging"...

# Checks

If you don't know the setting of your current network file, use the following cells to check it. 

In [22]:
# Which generators do exist?

n.generators

,carrier,bus,p_nom_min,p_nom,p_nom_extendable,efficiency,marginal_cost,capital_cost,build_year,lifetime,...,shut_down_cost,min_up_time,min_down_time,up_time_before,down_time_before,ramp_limit_up,ramp_limit_down,ramp_limit_start_up,ramp_limit_shut_down,p_nom_opt
Generator,,,,,,,,,,,,,,,,,,,,,
C29,OCGT,93,17.86,17.86,True,0.39,58.384615,47234.561404,1975,50.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
C30,OCGT,93,16.90,16.90,True,0.39,58.384615,47234.561404,1977,50.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
C31,OCGT,93,19.13,19.13,True,0.39,58.384615,47234.561404,1980,50.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
C32,OCGT,93,19.86,19.86,True,0.39,58.384615,47234.561404,1988,50.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
C33,OCGT,93,97.85,97.85,True,0.39,58.384615,47234.561404,1999,30.0,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21 hydro,ror,70,0.00,7.54,False,0.90,0.000000,270940.715283,0,inf,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
22 hydro,ror,55,0.00,11.49,False,0.90,0.000000,270940.715283,0,inf,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0
23 hydro,ror,55,0.00,5.15,False,0.90,0.000000,270940.715283,0,inf,...,0.0,0,0,1,0,NaN,NaN,1.0,1.0,0.0


In [38]:
n.generators.to_csv('C:/Users/noraky/Documents/Test_4/pypsa-earth/generators.csv')

In [23]:
# Which generators are currently extendable?

n.generators.p_nom_extendable


Generator
C29          True
C30          True
C31          True
C32          True
C33          True
            ...  
21 hydro    False
22 hydro    False
23 hydro    False
24 hydro    False
25 hydro    False
Name: p_nom_extendable, Length: 252, dtype: bool

In [31]:
n.generators.p_nom_extendable.to_csv('C:/Users/noraky/Documents/Test_4/pypsa-earth/generators_extendable.csv')

In [24]:
# Check the current status of the cp
n.generators.groupby('carrier').p_max_pu.mean()

carrier
CCGT       1.0
OCGT       1.0
biomass    1.0
oil        1.0
onwind     1.0
ror        1.0
solar      1.0
Name: p_max_pu, dtype: float64

In [25]:
# Which generators have which expansion limit?

n.generators.p_nom_max

Generator
C29         inf
C30         inf
C31         inf
C32         inf
C33         inf
           ... 
21 hydro    inf
22 hydro    inf
23 hydro    inf
24 hydro    inf
25 hydro    inf
Name: p_nom_max, Length: 252, dtype: float64

In [26]:
# Which links have which expansion limit?

n.links.p_nom_max

Link
0 battery charger         inf
3 battery charger         inf
5 battery charger         inf
6 battery charger         inf
9 battery charger         inf
                         ... 
99 battery discharger     inf
100 battery discharger    inf
101 battery discharger    inf
102 battery discharger    inf
103 battery discharger    inf
Name: p_nom_max, Length: 164, dtype: float64

In [27]:
# What is the sum of the demand in the network?

round(n.loads_t.p_set.sum().sum()/1000000,8)

9.45507556

In [28]:
# Is there any installed storage capacity?

n.stores.e_nom

Store
0 battery      0.0
3 battery      0.0
5 battery      0.0
6 battery      0.0
9 battery      0.0
              ... 
99 battery     0.0
100 battery    0.0
101 battery    0.0
102 battery    0.0
103 battery    0.0
Name: e_nom, Length: 82, dtype: float64

In [18]:
# What is the current emission limit?

n.global_constraints.constant.loc['CO2Limit']

77500000000.0

In [19]:
# Which stores have a maximum expansion limit?

n.stores.e_nom_max

Store
3 battery     inf
5 battery     inf
6 battery     inf
9 battery     inf
10 battery    inf
21 battery    inf
27 battery    inf
29 battery    inf
32 battery    inf
34 battery    inf
42 battery    inf
46 battery    inf
48 battery    inf
49 battery    inf
52 battery    inf
55 battery    inf
58 battery    inf
60 battery    inf
61 battery    inf
63 battery    inf
64 battery    inf
65 battery    inf
68 battery    inf
69 battery    inf
72 battery    inf
73 battery    inf
89 battery    inf
Name: e_nom_max, dtype: float64

In [20]:
n.buses

,bus_id,v_nom,tag_substation,tag_area,lon,lat,country,x,y,control,generator,carrier,type,unit,v_mag_pu_set,v_mag_pu_min,v_mag_pu_max,sub_network
Bus,,,,,,,,,,,,,,,,,,
3,3,220.0,transmission,97760.0,-67.0430,-17.9673,BO,-67.0430,-17.9673,PQ,,AC,,,1.0,0.0,inf,
5,5,220.0,transmission,20790.0,-65.7984,-19.6082,BO,-65.7984,-19.6082,PQ,,AC,,,1.0,0.0,inf,
6,6,220.0,transmission,20820.0,-63.6306,-17.4234,BO,-63.6306,-17.4234,PQ,,AC,,,1.0,0.0,inf,
9,9,220.0,transmission,41170.0,-63.2299,-17.7394,BO,-63.2299,-17.7394,PQ,,AC,,,1.0,0.0,inf,
10,10,220.0,transmission,13550.0,-66.2210,-17.5704,BO,-66.2210,-17.5704,Slack,10 onwind,AC,,,1.0,0.0,inf,
21,21,220.0,transmission,39140.0,-67.0374,-17.7129,BO,-67.0374,-17.7129,PQ,,AC,,,1.0,0.0,inf,
27,27,220.0,transmission,0.0,-67.1006,-17.8132,BO,-67.1006,-17.8132,PQ,,AC,,,1.0,0.0,inf,
29,29,220.0,transmission,15830.0,-65.6962,-19.5209,BO,-65.6962,-19.5209,PQ,,AC,,,1.0,0.0,inf,
32,32,220.0,transmission,57920.0,-63.1459,-17.5326,BO,-63.1459,-17.5326,PQ,,AC,,,1.0,0.0,inf,


# Restoring to 2021

When you reached your final year, you can easily restore your network to the year 2021. It is recommended, though, to save the initial 2021 file at the beginning. 

## Demand

In [5]:
year = 'back'
upscaling_factor = scale_demand[year]   # 0.569269674



n.loads_t.p_set = n.loads_t.p_set * upscaling_factor

year = 2021

NameError: name 'scale_demand' is not defined

In [66]:
n.loads_t.p_set.sum().sum()

5816726.662463184

## Emission limit

In [ ]:
n.global_constraints.constant = 1000000000 #emission_limit[2021]
n.global_constraints.constant

## Expansions

In [ ]:
solved_network = f'C:/Users/noraky/Documents/Test_4/pypsa-earth/results_NS/NS_2021.nc'
m = pypsa.Network(solved_network)

n.generators.p_nom = m.generators.p_nom
n.generators.p_nom

In [ ]:
n.generators.p_nom_min = m.generators.p_nom_min
n.generators.p_nom_min

In [ ]:
# new biomass to 0
for index, row in n.generators.iterrows():
    if index == 'BO 0 biomass':
        n.generators.at[index, 'p_nom'] = 0
    if index == 'BO 2 biomass':
            n.generators.at[index, 'p_nom'] = 0
n.generators.p_nom

In [ ]:
n.stores.e_nom = 0

n.stores.e_nom.sum()

In [ ]:
n.links.p_nom = 0
n.links.p_nom.sum()

# Changing the expandabel carriers

This can also be done in the config file, but doing it here saves you from running the model again and have to do all the other changes again as well.

### RES to TRUE

In [ ]:
# Iterate through the rows of the DataFrame
for index, row in n.generators.iterrows():
   
    if row['carrier'] == 'biomass':
        
        n.generators.at[index, 'p_nom_extendable'] = True 
    if row['carrier'] == 'solar':
        
        n.generators.at[index, 'p_nom_extendable'] = True 

    if row['carrier'] == 'onwind':
       
        n.generators.at[index, 'p_nom_extendable'] = True 

### Fossils to FALSE

In [ ]:
for index, row in n.generators.iterrows():
   
    if row['carrier'] == 'OCGT':
        n.generators.at[index, 'p_nom_extendable'] = False 

    if row['carrier'] == 'CCGT':
        
        n.generators.at[index, 'p_nom_extendable'] = False 

    if row['carrier'] == 'oil':
       
        n.generators.at[index, 'p_nom_extendable'] = False 

n.generators.p_nom_extendable